# Cifar-10 test for image classification

In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Resizing, Lambda, Flatten, Dense
import pandas as pd
import numpy as np
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

/Users/sasha/jax-metal/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Download RegNetX016 with Imagenet weights (1000 classes)

In [596]:
regnet = tf.keras.applications.regnet.RegNetX016(model_name='regnetx016',
                                                include_top=True,
                                                include_preprocessing=True,
                                                weights='imagenet',
                                                input_tensor=None,
                                                input_shape=None,
                                                pooling=None,
                                                classes=1000,
                                                classifier_activation='softmax'
                                                )

In [597]:
regnet.summary()

Model: "regnetx016"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_130 (InputLayer)      [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 regnetx016_prestem_rescali  (None, 224, 224, 3)          0         ['input_130[0][0]']           
 ng (Rescaling)                                                                                   
                                                                                                  
 regnetx016_stem_conv (Conv  (None, 112, 112, 32)         864       ['regnetx016_prestem_rescaling
 2D)                                                                [0][0]']                      
                                                                                         

Make all layers untrainable except for the very last convolutional layer

In [606]:
for layer in regnet.layers:
    layer.trainable = False
regnet.layers[-6].trainable  = True

Cifar-10 testing

In [607]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [608]:
y_train_cat = to_categorical(y_train, 1000)
y_test_cat = to_categorical(y_test, 1000)

Lambda layer for preprocessing

In [609]:
def resize(x):
    return tf.image.resize(x,size=(224,224),method='bicubic')

Modify the model

In [610]:
input_shape = (32,32,3)

In [611]:
input_layer = Input(shape=input_shape)
prep = Lambda(resize)(input_layer)
out = regnet(prep)
regnet_mod = Model(inputs=input_layer, outputs=out)

In [612]:
regnet_mod.compile(optimizer='adam',
                   loss=tf.keras.losses.CategoricalCrossentropy(),
                   metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1_categorical_accuracy')])

Try to fit it on Cifar-10 data and then evaluate (there is no hope this is gonna work ...)

In [613]:
regnet_mod.fit(X_train, y_train_cat)

1563/1563 [==============================] - 1521s 971ms/step - loss: 5.9514 - top_1_categorical_accuracy: 0.0000e+00


In [614]:
regnet_mod.evaluate(X_test, y_test_cat)

313/313 [==============================] - 409s 1s/step - loss: 8.6985 - top_1_categorical_accuracy: 0.0000e+00


[8.698457717895508, 0.0]

Try the same with adding a Cerebros "add-on" network

In [617]:
INPUT_SHAPES  = [input_shape]
OUTPUT_SHAPES = [10]

Use 10k-15k random samples from Cifar-10 to speed up the process

In [618]:
num_samples = 15_000
rng = np.random.default_rng()
ind = rng.permutation(X_train.shape[0])[:num_samples]

In [619]:
training_x   = [tf.constant(X_train[ind,:,:,:])]
y_train_cat  = to_categorical(y_train[ind], 10)
train_labels = [tf.constant(y_train_cat)]

In [620]:
regnet = tf.keras.applications.regnet.RegNetX016(model_name='regnetx016',
                                                include_top=True,
                                                include_preprocessing=True,
                                                weights='imagenet',
                                                input_tensor=None,
                                                input_shape=None,
                                                pooling=None,
                                                classes=1000,
                                                classifier_activation='softmax'
                                                )

In [622]:
for layer in regnet.layers:
    layer.trainable = False
regnet.layers[-6].trainable  = True

In [623]:
regnet_io = Model(inputs=regnet.layers[0].input,
                  outputs=regnet.layers[-2].output)

In [624]:
input_layer = Input(shape=input_shape)
prep = Lambda(resize)(input_layer)
out = Flatten()(regnet_io(prep))
base_mod = Model(inputs=input_layer, outputs=out)

In [626]:
activation = 'relu'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 10  # [1, 100]
batch_size = 20
maximum_levels = 4  # [3,7]
maximum_units_per_level = 7  # [2,10]
maximum_neurons_per_unit = 4  # [2,20]

In [627]:
# Final training task
TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
#
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test_cifar10_regnet'
#
meta_trial_number = 42
#
cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.2,
    direction='maximize',
    metric_to_rank_by="val_top_1_categorical_accuracy",
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation='softmax',
    number_of_architecture_moities_to_try=3,
    number_of_tries_per_architecture_moity=2,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.TopKCategoricalAccuracy(
                k=1, name='top_1_categorical_accuracy')
             ],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number,
    base_models=[base_mod])

In [628]:
%%time
result = cerebros_automl.run_random_search()

SimpleCerebrosRandomSearch.input_shapes: [(32, 32, 3)]
nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'1': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'10': <class 'cerebros.units.units.FinalDenseUnit'>}]
2
Trying to create Final level 2
Trying to create level 2
We think level final level 2's predecessors are: [0, 1]
levels:
[0, 1, 2]
{'0': 'InputUnitModule'}
InputLevel.input_shapes [(32, 32, 3)]
{'1': <class 'cerebros.units.units.DenseUnit'>}
{'3': <class 'cerebros.units.units.DenseUnit'>}
{'1': <class 'cerebros.units.units.DenseUnit'>}
{'2': <class 'cerebros.units.units.DenseUnit'>}
{'4': <class 'cerebros.units.units.DenseUnit'>}
Debug: I am 2 select

{'10': <class 'cerebros.units.units.FinalDenseUnit'>}
debug: meta_level_number
Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000002_tr_0_FinalDenseUnit_0000000000000002_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_3_dns_')>, <KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_3_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_4_dns_')>, <KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr

INFO:tensorflow:Assets written to: 2023_10_21_15_39_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.490705                    0.850583  0.397892   
1  0.285176                    0.904750  0.435407   
2  0.220730                    0.926583  0.446836   
3  0.167554                    0.940000  0.488218   
4  0.141045                    0.949583  0.412745   
5  0.105537                    0.963750  0.417840   
6  0.084596                    0.971667  0.525037   
7  0.088015                    0.967417  0.468035   
8  0.076636                    0.972833  0.524680   
9  0.071529                    0.976917  0.514112   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.861333             0                0   
1                        0.870000             0                0   
2                        0.875667             0                0   
3                        0.880333             0                0   
4                        0.888667             0   

materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000002_tr_0_FinalDenseUnit_0000000000000002_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_2_dns_')>, <KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_3_dns_')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_2_dns_')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_0_dns_')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_Den

...

INFO:tensorflow:Assets written to: 2023_10_21_15_39_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.104142                    0.957933  0.287475   
1  0.017591                    0.996000  0.323133   
2  0.011172                    0.997500  0.331279   
3  0.012682                    0.995833  0.418734   
4  0.013560                    0.995750  0.451277   
5  0.019442                    0.993333  0.480109   
6  0.012906                    0.996000  0.473203   
7  0.014645                    0.995083  0.528175   
8  0.017871                    0.993750  0.524280   
9  0.015223                    0.995500  0.579796   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.918333             0                0   
1                        0.923667             0                0   
2                        0.920667             0                0   
3                        0.908667             0                0   
4                        0.913000             0   

inputs
KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'), name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'")

outputs
KerasTensor(type_spec=TensorSpec(shape=(None, 10), dtype=tf.float32, name=None), name='NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000002_tr_0_FinalDenseUnit_0000000000000002_tr_0_0_dns_/Softmax:0', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000002_tr_0_FinalDenseUnit_0000000000000002_tr_0_0_dns_'")
Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
______________________________________________________________________

INFO:tensorflow:Assets written to: 2023_10_21_15_39_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000001/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.101878                    0.959933  0.298451   
1  0.013441                    0.997083  0.356516   
2  0.008856                    0.997833  0.332865   
3  0.010832                    0.996833  0.451827   
4  0.011395                    0.996667  0.432090   
5  0.010319                    0.997250  0.445817   
6  0.013825                    0.995333  0.523302   
7  0.015816                    0.995333  0.535993   
8  0.020776                    0.992583  0.519047   
9  0.012579                    0.995750  0.491757   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.920000             0                1   
1                        0.917333             0                1   
2                        0.921000             0                1   
3                        0.906667             0                1   
4                        0.916000             0   

In [630]:
print(f'Best accuracy achieved is {result}')
print(f'top-1 categorical accuracy')

Best accuracy achieved is 0.9236666560173036
top-1 categorical accuracy


Evaluating the best model found

In [632]:
best_model_found = cerebros_automl.get_best_model()

In [633]:
#
eval_loss = tf.keras.losses.CategoricalCrossentropy()
#
eval_metrics =\
[tf.keras.metrics.TopKCategoricalAccuracy(k=1,\
            name='eval_top_1_categorical_accuracy'),
 tf.keras.metrics.TopKCategoricalAccuracy(k=5,\
            name='eval_top_5_categorical_accuracy')
]

In [634]:
best_model_found.compile(loss=eval_loss, metrics=eval_metrics)
best_model_found.summary()

Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 NeuralNetworkFuture_000000  [(None, 32, 32, 3)]          0         []                            
 0000000nan_tr_0_InputLevel                                                                       
 _0000000000000000_tr_0_Inp                                                                       
 utUnit_0000000000000000_tr                                                                       
 _0_0_inp (InputLayer)                                                                            
                                                                                                  
 model_116 (Functional)      (None, 912)                  8320640   ['NeuralNetworkFuture_00000000
                                          

In [636]:
print("Evaluating best model found ...")
print("Loss | Top-1 accuracy | Top-5 accuracy")
y_test_cat = to_categorical(y_test, 10)
best_model_found.evaluate(X_test, y_test_cat)

Evaluating best model found ...
Loss | Top-1 accuracy | Top-5 accuracy
313/313 [==============================] - 481s 2s/step - loss: 0.6344 - eval_top_1_categorical_accuracy: 0.8978 - eval_top_5_categorical_accuracy: 0.9945


[0.6343686580657959, 0.8978000283241272, 0.9944999814033508]